In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.regularizers import l1

In [0]:
df_train = pd.read_csv('/content/WINE.csv')
df_test = pd.read_csv('/content/TEST.csv')

In [0]:
Y_train = np.array(df_train['Target'].astype('int32')).reshape(len(df_train['Target']),).T
df_train.drop(['Index', 'Target'], axis = 1, inplace = True)
df_test.drop(['Index'], axis = 1, inplace = True)

In [0]:
df_train.head()

,Type,Alcohol,Density,Nitrogen,pH,Sugar,A,B,C,D,Di,E,H,S,U,V
0,white_wine,7.50,1.02,7.51,3.19,10.85,25.1,1.98,10.2,39.6,49.10,1.37,2.36,2.34,6.35,0.29
1,white_wine,7.65,1.02,3.41,3.40,1.54,23.9,1.58,4.2,38.5,48.16,0.94,2.42,3.07,7.65,0.05
2,red_wine,8.10,1.02,0.25,3.76,2.11,21.7,7.92,0.3,22.0,34.93,1.26,4.84,4.38,4.40,0.22
3,white_wine,8.80,1.01,6.35,3.11,3.82,24.3,1.45,4.2,16.5,33.77,1.01,1.49,8.18,3.60,0.10
4,red_wine,7.80,1.02,5.34,3.51,3.25,27.3,5.15,4.5,31.9,40.26,1.57,4.15,4.53,6.45,0.14


In [0]:
df_train.loc[df_train['A'].isna(),'A'] = df_train['A'].mean()
df_train.loc[df_train['H'].isna(),'H'] = df_train['H'].mean()

In [0]:
one_hot = pd.get_dummies(df_train['Type'])
df_train.insert(loc = 1, column = 'red_wine', value = one_hot['red_wine'])
df_train.insert(loc = 2, column = 'white_wine', value = one_hot['white_wine'])
one_hot = pd.get_dummies(df_test['Type'])
df_test.insert(loc = 1, column = 'red_wine', value = one_hot['red_wine'])
df_test.insert(loc = 2, column = 'white_wine', value = one_hot['white_wine'])

In [0]:
df_train.drop(['Type'], axis = 1, inplace = True)
df_test.drop(['Type'], axis = 1, inplace = True)

In [0]:
df_train.head()

,red_wine,white_wine,Alcohol,Density,Nitrogen,pH,Sugar,A,B,C,D,Di,E,H,S,U,V
0,0,1,7.50,1.02,7.51,3.19,10.85,25.1,1.98,10.2,39.6,49.10,1.37,2.36,2.34,6.35,0.29
1,0,1,7.65,1.02,3.41,3.40,1.54,23.9,1.58,4.2,38.5,48.16,0.94,2.42,3.07,7.65,0.05
2,1,0,8.10,1.02,0.25,3.76,2.11,21.7,7.92,0.3,22.0,34.93,1.26,4.84,4.38,4.40,0.22
3,0,1,8.80,1.01,6.35,3.11,3.82,24.3,1.45,4.2,16.5,33.77,1.01,1.49,8.18,3.60,0.10
4,1,0,7.80,1.02,5.34,3.51,3.25,27.3,5.15,4.5,31.9,40.26,1.57,4.15,4.53,6.45,0.14


In [0]:
df_test.head()

,red_wine,white_wine,Alcohol,Density,Nitrogen,pH,Sugar,A,B,C,D,Di,E,H,S,U,V
0,0,1,9.25,1.01,5.72,3.47,2.58,24.5,1.98,5.85,30.8,48.30,1.29,1.49,4.67,4.80,0.06
1,0,1,7.45,1.02,7.04,3.20,10.76,25.1,1.72,10.20,40.7,55.05,1.05,2.23,2.04,6.35,0.05
2,0,1,9.10,1.01,11.82,3.30,3.06,23.7,2.90,5.25,9.9,22.11,1.29,1.80,4.53,7.90,0.03
3,1,0,7.90,1.02,3.96,3.51,2.11,24.9,7.39,1.05,7.7,18.07,1.09,5.64,5.26,0.85,0.38
4,0,1,7.65,1.02,6.76,3.27,12.65,24.7,2.64,8.10,67.1,85.00,1.26,2.48,3.36,7.83,0.18


In [0]:
standart_scaler = preprocessing.StandardScaler()
x = df_train.values
x_scaled = standart_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
X_train = np.array(df).reshape(len(df),len(df.columns))
x = df_test.values
x_scaled = standart_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
X_test = np.array(df).reshape(len(df),len(df.columns))

In [0]:
pca = PCA(n_components = 6)

In [0]:
princomp_train = pca.fit_transform(X_train)
princomp_test = pca.fit_transform(X_test)

In [0]:
principalDf_train = pd.DataFrame(data = princomp_train, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5', 'principal component 6'])
principalDf_test = pd.DataFrame(data = princomp_test, columns =['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5', 'principal component 6'])

In [0]:
principalDf_train.describe()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6
count,5.600000e+03,5.600000e+03,5.600000e+03,5.600000e+03,5.600000e+03,5.600000e+03
mean,-4.932562e-17,-1.871519e-17,-4.530106e-17,4.929836e-17,3.329492e-17,1.831868e-17
std,2.289506e+00,1.404488e+00,1.273471e+00,1.071084e+00,1.003039e+00,9.838508e-01
min,-7.257907e+00,-4.671640e+00,-5.494106e+00,-3.413042e+00,-1.387047e+00,-3.383485e+00
25%,-8.654801e-01,-9.990793e-01,-8.056906e-01,-7.242795e-01,-2.484583e-01,-6.313310e-01
50%,6.982829e-01,-1.064330e-01,-8.716316e-03,-7.982277e-03,-4.687805e-02,-1.541028e-01
75%,1.582608e+00,9.472923e-01,8.457363e-01,6.872926e-01,1.588833e-01,4.589918e-01
max,1.118819e+01,1.174394e+01,9.964398e+00,9.678337e+00,1.968893e+01,6.175933e+00


In [0]:
principalDf_test.describe()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6
count,7.900000e+02,7.900000e+02,7.900000e+02,7.900000e+02,7.900000e+02,7.900000e+02
mean,-1.337889e-16,-1.006227e-16,4.637640e-18,-2.782584e-17,-2.399628e-17,-3.435411e-17
std,2.308302e+00,1.564502e+00,1.299786e+00,1.113861e+00,9.887842e-01,9.764656e-01
min,-4.773639e+00,-4.355209e+00,-3.706346e+00,-3.060961e+00,-3.036164e+00,-3.126383e+00
25%,-1.611922e+00,-1.361727e+00,-8.319275e-01,-7.937444e-01,-6.770348e-01,-5.873274e-01
50%,-7.309817e-01,1.680994e-01,-4.023339e-02,-5.592826e-02,-7.888431e-02,4.601159e-02
75%,1.425035e+00,1.175917e+00,7.387064e-01,7.333403e-01,5.605126e-01,6.206691e-01
max,6.320431e+00,5.776725e+00,5.154362e+00,5.839455e+00,4.588531e+00,5.481795e+00


In [0]:
X_train = np.array(principalDf_train).astype('float32').T
X_test = np.array(principalDf_test).astype('float32').T

In [0]:
X_train.shape

(6, 5600)

In [0]:
Y_train.shape

(5600,)

In [0]:
def create_placeholders(n_x,n_y):
  X = tf.placeholder(tf.float32,[n_x,None], name = "X")
  Y = tf.placeholder(tf.float32,[n_y,None], name = "Y")
  return X,Y

In [0]:
def initialize_params(n_x):
  W = tf.get_variable("W", [6, n_x], initializer = tf.random_normal_initializer(seed = 1))
  b = tf.get_variable("b", [6,1], initializer = tf.zeros_initializer())
  params = {"W":W, "b":b}
  return params

In [0]:
def forward_prop(X, params):
  W = params['W']
  b = params["b"]
  Z = tf.add(tf.matmul(W,X), b)
  return Z

In [0]:
def to_one_hot(Y, C):
  C = tf.constant(C)
  one_hot_matrix = tf.one_hot(Y, depth=C, axis = 0)
  with tf.Session() as sess:
    one_hot = sess.run(one_hot_matrix)
  return one_hot

In [0]:
def cost_function(Z,Y, beta1, beta2, params):
  logits = tf.transpose(Z)
  labels = tf.transpose(Y)
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
  regulizer = params['W']
  cost = tf.reduce_mean(cost + beta1 * tf.reduce_sum(tf.abs(regulizer)))
  regulizer = tf.nn.l2_loss(params['W'])
  cost = tf.reduce_mean(cost + beta2 * regulizer)
  return cost

In [0]:
def predict(X, params):
  Z = forward_prop(X, params)
  return Z

In [0]:
def merge_model(X_train,Y_train, X_test, learning_rate, num_iterations, beta1, beta2):
  tf.reset_default_graph()
  Y_train = to_one_hot(Y_train, 6)
  (n_x, m) = X_train.shape
  n_y = Y_train.shape[0]
  X,Y = create_placeholders(n_x,n_y)
  costs = []
  iters = []
  params = initialize_params(n_x)
  Z = forward_prop(X, params)
  cost = cost_function(Z,Y, beta1, beta2, params)
  optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
  init = tf.global_variables_initializer()
  with tf.Session() as sess:
    sess.run(init)
    for iter in range(num_iterations+1):
      _ , cc = sess.run([optimizer, cost], feed_dict={X: X_train, Y: Y_train})
      if iter%100 == 0:
        costs.append(cc)
        iters.append(iter)
        print(cc)    
    prediction_train = tf.argmax(forward_prop(X_train, params))
    preds = prediction_train.eval()
    #prediction_test = tf.argmax(forward_prop(X_test, params))
    #preds_test = prediction_test.eval()
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    #print(correct_prediction)
    #print(prediction_train.eval())    
  return params, preds, preds_test

In [0]:
params, predicts, preds_test = merge_model(X_train,Y_train, X_test, 0.5, 1000, 0.05)

In [0]:
print(predicts.max())

4


In [0]:
preds_test

array([4, 3, 4, 4, 3, 4, 3, 3, 4, 4, 4, 3, 3, 4, 3, 4, 4, 4, 4, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 5, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 4, 4,
       4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3,
       3, 3, 4, 4, 4, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 4, 3, 4, 4, 3, 4, 4,
       4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 3, 3, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 3, 3, 4, 3, 3, 3, 5,
       4, 4, 4, 3, 5, 4, 3, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 3, 3, 4, 4,
       3, 3, 3, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 3, 3, 4,
       4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4,
       4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4,
       3, 4, 3, 4, 4, 4, 3, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 4, 3, 3, 4,
       4, 3, 5, 3, 3, 4, 4, 3, 4, 4, 4, 3, 3, 3, 4, 3, 4, 3, 4, 4, 4, 4,
       3, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4,

In [0]:
confusion_matrix(Y_train, predicts)

array([[   0,    0,   11,   14,    0,    0],
       [   0,    0,   73,  106,    4,    0],
       [   0,    0,  895,  960,    3,    0],
       [   0,    0,  599, 1812,   34,    0],
       [   0,    0,   92,  811,   28,    0],
       [   0,    0,   15,  137,    6,    0]])

In [0]:
concatenated_df = pd.read_csv('/content/Conc.csv')

In [0]:
concatenated_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [0]:
X_conc = np.array(concatenated_df).astype('float32').T
X_conc.shape

(9, 5600)

In [0]:
params, predicts, preds_test = merge_model(X_conc,Y_train, X_test, 0.01, 1500, 0.0001, 0.00005)

4.090156
1.3280565
1.0712249
1.0328194
1.0197091
1.013657
1.0105535
1.008938
1.0081701
1.0079085
1.0079846
1.0083228
1.0088911
1.0096437
1.0105681
1.0116433


In [0]:
confusion_matrix(Y_train, predicts)

array([[   0,    0,   11,   13,    1,    0],
       [   0,    0,   94,   87,    2,    0],
       [   0,    0, 1037,  808,   13,    0],
       [   0,    0,  612, 1723,  110,    0],
       [   0,    1,   85,  717,  128,    0],
       [   0,    0,   18,  104,   36,    0]])

In [0]:
confusion_matrix(Y_train, predicts)

array([[   1,    0,   13,   10,    1,    0],
       [   0,    4,   90,   87,    2,    0],
       [   0,    1, 1097,  747,   13,    0],
       [   0,    0,  646, 1657,  142,    0],
       [   0,    0,   93,  675,  163,    0],
       [   0,    0,   18,   96,   44,    0]])

In [0]:
confusion_matrix(Y_train, predicts)

array([[   0,    0,   11,   13,    1,    0],
       [   0,    2,   87,   92,    2,    0],
       [   0,    1, 1078,  767,   12,    0],
       [   0,    0,  681, 1628,  136,    0],
       [   0,    0,  101,  670,  160,    0],
       [   0,    0,   18,   86,   54,    0]])

In [0]:
f1_score(Y_train, predicts, average = 'weighted')

0.47768603442044205

In [0]:
precision_score(Y_train, predicts, average = 'weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.4939705526989233

In [0]:
recall_score(Y_train, predicts, average = 'weighted')

0.5121428571428571